In [11]:
import pandas as pd
import jupyterlab
import matplotlib.pyplot as plt
import os

path = os.getcwd()
files = os.listdir(path)
# gets all xlsx files in cwd
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
df = pd.DataFrame()
for file in files_xlsx:
    data = pd.read_excel(file, 'peptide')
    df = df.append(data)  # appends all data

df = df.loc[(df['#Feature'] >= 4)] # sorts out if feature<4
#df["F6"] = df["#Feature"].apply(lambda x: x > 6), df.loc[df['F6]]
df['count'] = 1

df = df.drop(['-10lgP', 'ppm', 'm/z', 'Area WF1', 'Area WF2', 'Area WF 3', 'Area WF4', 'Area WF5'], axis=1)
df = df.groupby(['Peptide', 'Accession']).sum()
df['count*feat'] = df['count'] * df['#Feature']
df['cutsite'] = df['Peptide'].apply(lambda x: x[0:4])
df = df.sort_values(['count*feat'], ascending=False)

In [10]:
df

,,Mass,Length,RT,Fraction,#Feature,#Feature WF1,#Feature WF2,#Feature WF 3,#Feature WF4,#Feature WF5,count,count*feat
Peptide,Accession,,,,,,,,,,,,
PSLQAPVR,sp|Q04206|TF65_HUMAN,4332.4870,40,74.40,52,160,23.0,8.0,38.0,52.0,38.0,5,800
AVLTIDEKGTEAAGAM(+15.99)F,sp|P01009|A1AT_HUMAN,6955.3788,68,270.21,47,103,6.0,0.0,47.0,12.0,38.0,4,412
NFLSYFVELGTQPATQ,sp|P02652|APOA2_HUMAN,9069.4435,80,589.24,52,55,11.0,14.0,9.0,9.0,12.0,5,275
GEYGAEALERM(+15.99)FL,sp|P69905|HBA_HUMAN,7503.4590,65,495.22,55,46,8.0,10.0,10.0,8.0,10.0,5,230
HAGEYGAEALERM(+15.99)FL,sp|P69905|HBA_HUMAN,8543.9390,75,394.46,53,45,9.0,13.0,11.0,5.0,7.0,5,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GVQIPLPEGI,sp|P55058|PLTP_HUMAN,1021.5807,10,103.95,8,4,0.0,4.0,0.0,0.0,0.0,1,4
GVEQDVDM(+15.99)VF,sp|P14618|KPYM_HUMAN,1153.4961,10,87.17,8,4,0.0,4.0,0.0,0.0,0.0,1,4
SFLSALEEYTKKLN,sp|P02647|APOA1_HUMAN,1641.8613,14,105.78,3,4,4.0,0.0,0.0,0.0,0.0,1,4
